In [1]:
import json
import os
import pandas as pd
import numpy as np
import glob

DREBIN_FEATURE_PATH = "/scratch1/NOT_BACKED_UP/cavallarogrp/datasets/processed_dataset/Androzoo_info/drebin_22_23"
ANDROZOO_INFO_PATH = "/cs/academic/phd3/xinrzhen/xinran/invariant_training/raw_dataset/androzoo_family_with_gray"
ANDROZ00_INFO = "/scratch1/NOT_BACKED_UP/cavallarogrp/datasets/processed_dataset/Androzoo_info"


In [2]:
df_meta = pd.read_csv(os.path.join(ANDROZ00_INFO, 'final_meta_info_file.tsv'), delimiter='\t')
print(df_meta.head())
print(df_meta.columns)

                                              sha256             dex_date  \
0  0000143ef8d00e3a65c5c8c380221d00678fed906fdc2e...  2016-01-19 13:28:36   
1  0000205479c9ba53e1b209d5fc484a8b5d12235b83374e...  2016-05-03 17:25:52   
2  0000230ae0799eaa963f0ae5337a0e6e1ac32acf1fa4e3...  2014-05-15 19:23:12   
3  0000241b6c550b52927c74c854b7a8d9dba6e86a3576d5...  2015-11-09 17:06:28   
4  000026e846ffcf60e2a9e135251b4f411300e903dce95d...  2015-09-13 21:46:06   

   label  family  
0    0.0  benign  
1    0.0  benign  
2    0.0  benign  
3    0.0  benign  
4    1.0     NaN  
Index(['sha256', 'dex_date', 'label', 'family'], dtype='object')


In [5]:
drebin_14_21_path = os.path.join(ANDROZ00_INFO, "drebin_14_21.json")
with open(drebin_14_21_path, "r") as f:
    drebin_14_21 = json.load(f)

In [2]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from collections import Counter

years = ['2022', '2023']
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

output_directory = '/scratch1/NOT_BACKED_UP/cavallarogrp/datasets/feature_space/features/drebin_22_23'
info_22_23 = '/cs/academic/phd3/xinrzhen/xinran/invariant_training/data_process/22_23_sha256.txt'

df_info = pd.read_csv(info_22_23, header=None, names=["sha256","dex_date","vt_detection"])

def find_vt_detection(sha256):
    vt_detection = df_info[df_info['sha256'] == sha256]['vt_detection'].values
    if len(vt_detection) > 0:
        return vt_detection[0]
    else:
        return 0

# 使用并行读取JSON文件
def load_feature(sha256,name):
    feature_path = f"{DREBIN_FEATURE_PATH}/{name}/{sha256}.json"
    if os.path.exists(feature_path):  # 确保文件存在
        with open(feature_path, 'r') as f:
            feature = json.load(f)
        return feature.get(sha256, None)
    

def delete_feature(sha256, name):
    feature_path = f"{DREBIN_FEATURE_PATH}/{name}/{sha256}.json"
    if os.path.exists(feature_path):
        os.remove(feature_path)

save_folder = os.path.join(DREBIN_FEATURE_PATH, "features")
fail_list = []
# 读取CSV文件
df_all = []
for year in years:
    for month in months:
        name = f"{year}-{month}"
        print(f"Processing {name}")
        month_output_dir = os.path.join(output_directory, name)
        if not os.path.exists(month_output_dir):
            os.makedirs(month_output_dir)

        df = pd.read_csv(f"{ANDROZOO_INFO_PATH}/{year}-{month}.txt", header=None, names=["apk_name", "label", "releaseTime" , "family_label"])
        # df = pd.read_csv(f"{ANDROZOO_INFO_PATH}/{year}-{month}.txt")
        # df_meta = df[["sha256", "date", "label", "family"]]
        # df_meta.columns = ["sha256", "dex_date", "label", "family"]
        df_meta = df[["apk_name", "label", "releaseTime" , "family_label"]]
        df_meta['vt_detection'] = 0
        df_meta['vt_detection'] = df_meta['apk_name'].apply(find_vt_detection)
        print(Counter(df_meta['vt_detection']))
        print(Counter(df_meta['label']))
        df_meta.columns = ["sha256",  "label", "dex_date", "family","vt_detection"]
        if not os.path.exists(os.path.join(f"{save_folder}/{year}-{month}")):
            os.makedirs(os.path.join(f"{save_folder}/{year}-{month}"))
        df_meta.to_csv(f"{save_folder}/{year}-{month}/{year}-{month}-meta.csv", index=False)
        sha256s = df['apk_name'].values.tolist()
        print(f"Total {len(sha256s)} samples in read txt")

        existed_files = glob.glob(f"{DREBIN_FEATURE_PATH}/{name}/*.json")
        existed_sha256 = [os.path.basename(file).split(".")[0] for file in existed_files]
        difference = set(sha256s) - set(existed_sha256)
        sha256s = list(set(sha256s) - difference)
        print(f"Total {len(difference)} difference files")
        print(difference)
        for sha256 in difference:
            delete_feature(sha256, name)
            print(f"Delete {sha256}")

        X = []

        existed_files = os.listdir(os.path.join(save_folder, f"{name}"))

        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(load_feature, sha256, name) for sha256 in sha256s]
            for future in tqdm(as_completed(futures), total=len(futures)):
                try:
                    feature = future.result()
                except Exception as e:
                    print(e)
                    fail_list.append(sha256s[futures.index(future)])
                    feature = None
                
                if feature is not None:
                    X.append(feature)

        sha256s_final = [sha256 for sha256 in sha256s if sha256 not in fail_list]
        print(f"fail list: {fail_list}")
        print(f"fail list length: {len(fail_list)}, final length: {len(sha256s_final)}")
        print(f"Total {len(X)} samples")

        assert len(X) == len(sha256s_final)

        df_meta = df_meta[df_meta['sha256'].isin(sha256s_final)]

        os.makedirs(os.path.join(save_folder, f"{name}"), exist_ok=True)
        save_path = os.path.join(save_folder,f"{name}",f"{name}.json")
        with open(save_path, "w") as f:
            json.dump(X, f)
        df_meta['json_features'] = X
        df_meta['dex_date'] = pd.to_datetime(df_meta['dex_date'], errors='coerce')
        print(f"final saved shape: {df_meta.shape}")
        file_name = 'features.pkl'
        file_path = os.path.join(month_output_dir, file_name)

        df_meta.to_pickle(file_path)
        print(f"Save to {file_path}")
        df_all.append(df_meta)

df_all = pd.concat(df_all)
# print(f"Fail list: {len(fail_list)}")

# for sha256 in tqdm(fail_list):
#     delete_json_path = f"{DREBIN_FEATURE_PATH}/{year}-{month}/{sha256}.json"
#     print(f"Delete {delete_json_path}")
#     os.remove(delete_json_path)
    

Processing 2022-01
Counter({0.0: 1379, 5.0: 58, 6.0: 46, 7.0: 24, 8.0: 12, 10.0: 4, 9.0: 2, 17.0: 1, 13.0: 1, 12.0: 1})
Counter({0: 1379, 1: 149})
Total 1528 samples in read txt
Total 0 difference files
set()


100%|██████████| 1528/1528 [00:01<00:00, 1165.81it/s]


fail list: []
fail list length: 0, final length: 1528
Total 1528 samples
final saved shape: (1528, 6)
Save to /scratch1/NOT_BACKED_UP/cavallarogrp/datasets/feature_space/features/drebin_22_23/2022-01/features.pkl
Processing 2022-02
Counter({0.0: 1407, 5.0: 58, 6.0: 41, 7.0: 33, 8.0: 12, 9.0: 4, 12.0: 2, 17.0: 1, 11.0: 1})
Counter({0: 1407, 1: 152})
Total 1559 samples in read txt
Total 0 difference files
set()


100%|██████████| 1559/1559 [00:00<00:00, 1819.82it/s]


fail list: []
fail list length: 0, final length: 1559
Total 1559 samples
final saved shape: (1559, 6)
Save to /scratch1/NOT_BACKED_UP/cavallarogrp/datasets/feature_space/features/drebin_22_23/2022-02/features.pkl
Processing 2022-03
Counter({0.0: 1379, 5.0: 63, 6.0: 47, 7.0: 22, 9.0: 5, 8.0: 5, 12.0: 3, 10.0: 2, 11.0: 1, 19.0: 1})
Counter({0: 1379, 1: 149})
Total 1528 samples in read txt
Total 0 difference files
set()


...

Expecting value: line 1 column 1 (char 0)


100%|██████████| 567/567 [00:00<00:00, 889.23it/s] 


fail list: ['4D052FD84FF96D5325DD2AAEE6C03DAEA0D2009531E07284B8E11BFB56D78A36', 'B1FB0B082189A388D80DA64BF1846A3CDEA7325B5F9979F17411E432E223302A', '6C91B5B97D0BE4476B5D930324FB5B4CEF64D76B9B829A0E7054CFC94B4E6569', 'ADB918706343143C3041BCFD76301F552F6B4E9F485F414E994F022BAC2A6804', '12F56A5617BC8C8639366D92A51445C1B6A7F47C1B71BD72DDF93D6A22917C5E', 'E5E5571B77C6F758F81946F556C19383728CCB840C372AC09294EF9A82DFD494']
fail list length: 6, final length: 566
Total 566 samples
final saved shape: (566, 6)
Save to /scratch1/NOT_BACKED_UP/cavallarogrp/datasets/feature_space/features/drebin_22_23/2023-11/features.pkl
Processing 2023-12
Counter({0.0: 833, 5.0: 59, 6.0: 18, 7.0: 5, 8.0: 3, 27.0: 1, 10.0: 1, 11.0: 1, 9.0: 1, 12.0: 1})
Counter({0: 833, 1: 90})
Total 923 samples in read txt
Total 380 difference files
{'...'}
Delete 1A2FBFBD825B516E9BD498B217564F2FC05561D77B792F0D1DA73AA40353F3FB
Delete 8E917CD230DC32B0B4A25F3574310DD93479C6155305D8A6299FF5CA733548A9
Delete DE510F2233D3CBE76A0E77E6A0

100%|██████████| 543/543 [00:00<00:00, 881.01it/s]


fail list: ['4D052FD84FF96D5325DD2AAEE6C03DAEA0D2009531E07284B8E11BFB56D78A36', 'B1FB0B082189A388D80DA64BF1846A3CDEA7325B5F9979F17411E432E223302A', '6C91B5B97D0BE4476B5D930324FB5B4CEF64D76B9B829A0E7054CFC94B4E6569', 'ADB918706343143C3041BCFD76301F552F6B4E9F485F414E994F022BAC2A6804', '12F56A5617BC8C8639366D92A51445C1B6A7F47C1B71BD72DDF93D6A22917C5E', 'E5E5571B77C6F758F81946F556C19383728CCB840C372AC09294EF9A82DFD494']
fail list length: 6, final length: 543
Total 543 samples
final saved shape: (543, 6)
Save to /scratch1/NOT_BACKED_UP/cavallarogrp/datasets/feature_space/features/drebin_22_23/2023-12/features.pkl


In [28]:
for sha256 in difference:
    delete_json_path = f"/scratch1/NOT_BACKED_UP/cavallarogrp/datasets/feature_space/raw_smali/{sha256}"
    print(f"Delete {delete_json_path}")
    if os.path.exists(delete_json_path):
        os.remove(delete_json_path)
    else:
        print(f"{delete_json_path} not exists")

Delete /scratch1/NOT_BACKED_UP/cavallarogrp/datasets/feature_space/raw_smali/2AE8EA1BAAEC8C5CA09118F10458BDA7790A7D591A0E81E32F7B11AFE522B1BE
/scratch1/NOT_BACKED_UP/cavallarogrp/datasets/feature_space/raw_smali/2AE8EA1BAAEC8C5CA09118F10458BDA7790A7D591A0E81E32F7B11AFE522B1BE not exists
Delete /scratch1/NOT_BACKED_UP/cavallarogrp/datasets/feature_space/raw_smali/1950B1D8A95D5F9C148983ADABD398ECC506053E1AC66C03B3F5500E5847B177
/scratch1/NOT_BACKED_UP/cavallarogrp/datasets/feature_space/raw_smali/1950B1D8A95D5F9C148983ADABD398ECC506053E1AC66C03B3F5500E5847B177 not exists
Delete /scratch1/NOT_BACKED_UP/cavallarogrp/datasets/feature_space/raw_smali/1983368C0EFE4C312A4322A2DEBF8E117A6D0FA87A50FBDA431A265A1654D153
/scratch1/NOT_BACKED_UP/cavallarogrp/datasets/feature_space/raw_smali/1983368C0EFE4C312A4322A2DEBF8E117A6D0FA87A50FBDA431A265A1654D153 not exists
Delete /scratch1/NOT_BACKED_UP/cavallarogrp/datasets/feature_space/raw_smali/9501C24E6BDC96315086469B8FD0D794E2C0E3A37046B713D77AA70F89F

In [26]:
for sha256 in fail_list:
    delete_feature(sha256, name)
    print(f"Delete {sha256}")

Delete 7A77335464456B38847D6EF835E986EC88B83153EAFB5464DEE5335CFA0DF379


In [7]:
from sklearn.feature_extraction import DictVectorizer

vec =  DictVectorizer()
X = vec.fit_transform(X)
print(X.shape)

(1528, 114847)
